In [86]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
import traceback

In [87]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [88]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [89]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GEMINI_API_KEY)

In [90]:
from langchain_core.prompts import PromptTemplate

In [91]:
TEMPLATE = """
Text : {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
###RESPONSE JSON
{response_json}"

"""

In [92]:
RESPONSE_JSON = {
    "1":  {
        "MCQ": "Multiple Choice Question",
        "Options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"},
        "Correct": "Correct Answer"
    },
    "2":  {
        "MCQ": "Multiple Choice Question",
        "Options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"},
        "Correct": "Correct Answer"
    },
    "3":  {
        "MCQ": "Multiple Choice Question",
        "Options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"},
        "Correct": "Correct Answer"
    },
}

In [93]:
quiz_prompt = PromptTemplate(template=TEMPLATE, input_variables=[
                             "text", "number", "subject", "tone", "response_json"])

In [94]:
from langchain.chains import LLMChain

quiz_chain = LLMChain(llm=llm, prompt=quiz_prompt,
                      output_key="quiz", verbose=True)

In [95]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [96]:
evaluation_prompt = PromptTemplate(
    template=TEMPLATE2, input_variables=["subject", "quiz"])

In [97]:
review_chain = LLMChain(llm=llm, prompt=evaluation_prompt,
                        output_key="review", verbose=True)

In [98]:
from langchain.chains import SequentialChain

generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=[
    "text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [99]:
with open("D:\GenerativeAI\MCQGenerator\data.txt", "r") as f:
    data = f.read()

In [100]:
print(data)

Machine learning (ML) is a branch of artificial intelligence (AI) and computer science that focuses on the using data and algorithms to enable AI to imitate the way that humans learn, gradually improving its accuracy.

How does machine learning work?
UC Berkeley (link resides outside ibm.com) breaks out the learning system of a machine learning algorithm into three main parts.

A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.
An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.
A Model Optimization Process: If the model can fit better to the data points in the training set, then weights are adjusted to reduce the discrepancy between the known example and the model estimate

In [101]:
json.dumps(RESPONSE_JSON)

'{"1": {"MCQ": "Multiple Choice Question", "Options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "Correct": "Correct Answer"}, "2": {"MCQ": "Multiple Choice Question", "Options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "Correct": "Correct Answer"}, "3": {"MCQ": "Multiple Choice Question", "Options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "Correct": "Correct Answer"}}'

In [102]:
with open("D:\GenerativeAI\MCQGenerator\Response.json", "r") as f:
    response_json = json.load(f)

In [103]:
NUMBER = 5
SUBJECT = "Data Science"
TONE = "Simple"

In [104]:
response = generate_evaluate_chain(
    {"text": data, "number": NUMBER, "subject": SUBJECT, "tone": TONE, "response_json": RESPONSE_JSON})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text : Machine learning (ML) is a branch of artificial intelligence (AI) and computer science that focuses on the using data and algorithms to enable AI to imitate the way that humans learn, gradually improving its accuracy.

How does machine learning work?
UC Berkeley (link resides outside ibm.com) breaks out the learning system of a machine learning algorithm into three main parts.

A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.
An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.
A Model Optimization Process: If the model can fit better to the data points in the trai

In [105]:
type(response)

dict

In [116]:
quiz = response.get("quiz")
quiz = quiz.split('\n', 1)[1]
quiz = quiz.replace("'", "\"")
quiz = json.loads(quiz)

In [117]:
quiz

{'1': {'MCQ': 'Which of the following is NOT a part of a machine learning algorithm learning system?',
  'Options': {'a': 'Decision Process',
   'b': 'Error Function',
   'c': 'Data Preparation',
   'd': 'Model Optimization Process'},
  'Correct': 'c'},
 '2': {'MCQ': 'What type of machine learning uses labeled datasets and can automatically determine the set of features that distinguish different categories of data?',
  'Options': {'a': 'Supervised learning',
   'b': 'Unsupervised learning',
   'c': 'Deep learning',
   'd': 'Reinforcement learning'},
  'Correct': 'c'},
 '3': {'MCQ': 'Which of the following algorithms is used for predicting numerical values based on a linear relationship between different values?',
  'Options': {'a': 'Neural networks',
   'b': 'Linear regression',
   'c': 'Logistic regression',
   'd': 'Clustering'},
  'Correct': 'b'},
 '4': {'MCQ': 'What is the term for the capability that uses natural language processing to translate human speech into a written format

In [118]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["MCQ"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["Options"].items()
        ]
    )
    correct = value["Correct"]
    quiz_table_data.append(
        {"MCQ": mcq, "Choices": options, "Correct": correct})

In [119]:
quiz_table_data

[{'MCQ': 'Which of the following is NOT a part of a machine learning algorithm learning system?',
  'Choices': 'a: Decision Process | b: Error Function | c: Data Preparation | d: Model Optimization Process',
  'Correct': 'c'},
 {'MCQ': 'What type of machine learning uses labeled datasets and can automatically determine the set of features that distinguish different categories of data?',
  'Choices': 'a: Supervised learning | b: Unsupervised learning | c: Deep learning | d: Reinforcement learning',
  'Correct': 'c'},
 {'MCQ': 'Which of the following algorithms is used for predicting numerical values based on a linear relationship between different values?',
  'Choices': 'a: Neural networks | b: Linear regression | c: Logistic regression | d: Clustering',
  'Correct': 'b'},
 {'MCQ': 'What is the term for the capability that uses natural language processing to translate human speech into a written format?',
  'Choices': 'a: Automatic speech recognition | b: Computer vision | c: Recommenda

In [120]:
mcq_data = pd.DataFrame(quiz_table_data)

In [121]:
mcq_data

,MCQ,Choices,Correct
0,Which of the following is NOT a part of a mach...,a: Decision Process | b: Error Function | c: D...,c
1,What type of machine learning uses labeled dat...,a: Supervised learning | b: Unsupervised learn...,c
2,Which of the following algorithms is used for ...,a: Neural networks | b: Linear regression | c:...,b
3,What is the term for the capability that uses ...,a: Automatic speech recognition | b: Computer ...,a
4,Which ethical concern of machine learning invo...,a: Privacy | b: Accountability | c: Bias and d...,c


In [ ]:
mcq_data.to_csv("mcq_sample_data.csv", index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'08_16_2024_10_07_39'